<a href="https://colab.research.google.com/github/muffindonor/cloud_computing_project/blob/main/Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# התקנת ספריית Firebase Admin
!pip install firebase

# התקנת חבילות נדרשות במידת הצורך
!pip install firebase beautifulsoup4

In [33]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
from urllib.parse import urljoin
from firebase import firebase
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords


class SearchEngine:
    def __init__(self):
        """Initialize the search engine"""
        self.pages = []
        self.word_locations = defaultdict(list)  # word -> [(page_id, frequency), ...]
        self.stop_words = set(stopwords.words('english'))
        self.firebase_conn = firebase.FirebaseApplication('https://cloudproject-27420-default-rtdb.firebaseio.com/', None)



    def fetch_pages(self, base_url, num_pages=5):
        """Fetch pages from the provided base URL"""
        try:
            response = requests.get(base_url)
            soup = BeautifulSoup(response.text, 'html.parser')

            links = []
            for a_tag in soup.find_all('a', href=True):
                full_url = urljoin(base_url, a_tag['href'])
                if full_url.startswith(base_url):
                    links.append(full_url)

            links = list(set(links))[:num_pages]
            print(f"Found {len(links)} links. Processing the first {num_pages}.")

            for link in links:
                try:
                    page_response = requests.get(link)
                    page_soup = BeautifulSoup(page_response.text, 'html.parser')

                    # Extract text from all visible elements
                    for script in page_soup(["script", "style", "meta", "link"]):
                        script.decompose()

                    page_text = page_soup.get_text(" ", strip=True)

                    self.pages.append({
                        'id': link,
                        'title': page_soup.title.string if page_soup.title else 'No Title',
                        'url': link,
                        'content': page_text
                    })
                    print(f"Retrieved: {link}")
                except Exception as e:
                    print(f"Error processing {link}: {str(e)}")
                    continue

            return True

        except Exception as e:
            print(f"Error fetching pages: {str(e)}")
            return False



    def merge_index_data(self, existing_data, new_data):
        """Merge existing index data with new data"""
        merged = defaultdict(list)

        # Convert existing data to dictionary for easier lookup
        if existing_data:
            for entry in existing_data:
                merged[entry['Term']] = entry['DocIDs']

        # Merge new data
        for term, locations in new_data.items():
            existing_locations = merged[term]
            # Check for duplicate DocIDs
            existing_doc_ids = {loc['DocID'] for loc in existing_locations}

            # Add only new locations
            for new_loc in locations:
                if new_loc['DocID'] not in existing_doc_ids:
                    existing_locations.append(new_loc)
                    existing_doc_ids.add(new_loc['DocID'])

            merged[term] = existing_locations

        return [{'Term': term, 'DocIDs': locations} for term, locations in merged.items()]

    def build_index(self):
        """Build or update the word location index"""
        # Clear local index before building new one
        self.word_locations.clear()

        for page in self.pages:
            content = page['content']
            word_counts = defaultdict(int)

            # Process content for each word
            for word in re.findall(r'\b\w+\b', content):
                word_lower = word.lower()
                if word_lower not in self.stop_words:
                    word_counts[word_lower] += 1

            # Process compound words
            for compound in re.finditer(r'\b[A-Z][a-zA-Z]*[A-Z][a-zA-Z]*\b', content):
                word = compound.group()
                parts = re.findall(r'[A-Z][a-z]*', word)
                for part in parts:
                    part_lower = part.lower()
                    if part_lower not in self.stop_words:
                        word_counts[part_lower] += 1

            # Add counts to index
            for word, count in word_counts.items():
                self.word_locations[word].append({
                    'DocID': page['id'],
                    'Title': page['title'],
                    'Frequency': count
                })

        print("Local index built successfully!")

        existing_index = self.firebase_conn.get('/', 'word_index')
        new_index_data = {term: locations for term, locations in self.word_locations.items()}
        merged_index = self.merge_index_data(existing_index, new_index_data)
        self.firebase_conn.put('/', 'word_index', merged_index)
        print("Index updated in Firebase successfully!")

    def search(self, query, num_results=5):
        """Search for a query in the indexed data"""
        query_words = [word.lower() for word in re.findall(r'\w+', query) if word.lower() not in self.stop_words]
        if not query_words:
            return []

        page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})
        # Get matches for both exact query and compound words containing query
        for entry in self.word_locations.get(query, []):
            page_id = entry['DocID']
            page_scores[page_id]['matches'] += 1
            page_scores[page_id]['total_freq'] += entry['Frequency']

        ranked_results = sorted(
            [(page_id, scores['matches'], scores['total_freq']) for page_id, scores in page_scores.items()],
            key=lambda x: (x[1], x[2]), reverse=True
        )

        results = []
        for page_id, matches, total_freq in ranked_results[:num_results]:
            page = next((p for p in self.pages if p['id'] == page_id), None)
            if page:
                results.append({
                    'title': page['title'],
                    'url': page['url'],
                    'matching_words': matches,
                    'total_frequency': total_freq
                })
        return results


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
# Logic for search and result screen
from firebase import firebase

class SearchLogic:
    def __init__(self, firebase_url):
        self.firebase_conn = firebase.FirebaseApplication(firebase_url, None)
        self.index_data = self.fetch_index()

    def fetch_index(self):
        try:
            index = self.firebase_conn.get('/word_index', None)
            return index if index else []
        except Exception as e:
            print(f"Error fetching index from Firebase: {e}")
            return []

    def search_query(self, query):
        query_words = [word.lower() for word in query.split()]
        results = []
        for term in query_words:
            for entry in self.index_data:
                if entry['Term'] == term:
                    for occurrence in entry['DocIDs']:
                        results.append({
                            'page_id': occurrence['DocID'],
                            'title': occurrence['Title'],
                            'frequency': occurrence['Frequency']
                        })
        return results


In [35]:
# Style for search and result screen
import ipywidgets as widgets
from IPython.display import display, clear_output

class SearchUI:
    def __init__(self, logic):
        self.logic = logic

    def show_search_screen(self):
        clear_output()

        background_style = '''
        <style>
            body {
                background-color: #1a202c !important;
                min-height: 100vh;
                margin: 0;
                padding: 20px;
                box-sizing: border-box;
            }
            .widget-button button {
                background: linear-gradient(90deg, #ef4444 0%, #f59e0b 100%) !important;
                border: none !important;
                color: white !important;
                border-radius: 20px !important;
                padding: 8px 25px !important;
                font-weight: bold !important;
                transition: transform 0.2s !important;
                box-shadow: none !important;
            }
            .widget-button button:hover {
                transform: scale(1.05) !important;
            }
            .widget-text input {
                background-color: #2d3748 !important;
                border: none !important;
                color: white !important;
                border-radius: 9999px !important;
                padding: 15px 25px !important;
                width: 100% !important;
            }
            .widget-text input::placeholder {
                color: #718096 !important;
            }
            .admin-button {
                background-color: #3b82f6 !important;
                color: white !important;
                padding: 8px 16px !important;
                border-radius: 6px !important;
                border: none !important;
                cursor: pointer !important;
                position: absolute !important;
                right: 20px !important;
                top: 20px !important;
            }
        </style>
        '''

        logo_html = '''
        <div style="text-align: center; margin: 40px auto;">
            <div style="width: 150px; height: 150px; margin: 0 auto; position: relative;">
                <div style="
                    width: 150px;
                    height: 150px;
                    border-radius: 50%;
                    background: white;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    position: relative;
                    z-index: 1;
                ">
                    <span style="color: #718096; font-size: 16px;">150 × 150</span>
                </div>
                <div style="
                    position: absolute;
                    top: -4px;
                    left: -4px;
                    right: -4px;
                    bottom: -4px;
                    background: linear-gradient(90deg, #ef4444 0%, #f59e0b 50%, #ef4444 100%);
                    border-radius: 50%;
                    z-index: 0;
                "></div>
            </div>
        </div>
        '''

        # Admin button
        admin_button = widgets.Button(
            description="Admin Page",
            button_style="warning",
            layout=widgets.Layout(width="auto")
        )

        # Search bar and button
        search_bar = widgets.Text(
            placeholder="Search...",
            layout=widgets.Layout(width="400px")
        )

        search_button = widgets.Button(
            description="Badger Search",
            button_style="warning",
            layout=widgets.Layout(width="auto")
        )

        # Define handlers
        def on_admin_clicked(b):
            self.show_admin_page()

        def on_search_clicked(b):
            query = search_bar.value.strip()
            if query:
                results = self.logic.search_query(query)
                self.show_results_screen(query, results)
            else:
                output = widgets.HTML("<div style='color: #ef4444; text-align: center; margin-top: 10px;'>Please enter a valid query!</div>")
                display(output)

        # Attach handlers
        admin_button.on_click(on_admin_clicked)
        search_button.on_click(on_search_clicked)

        # Display elements
        display(widgets.HTML(background_style))
        display(admin_button)
        display(widgets.HTML(logo_html))

        search_container = widgets.VBox([search_bar, search_button],
            layout=widgets.Layout(
                display='flex',
                flex_flow='column',
                align_items='center',
                width='100%',
                max_width='700px',
                margin='0 auto'
            )
        )
        display(search_container)

    def show_admin_page(self):
        clear_output()

        background_style = '''
        <style>
            body {
                background-color: #1a202c !important;
                min-height: 100vh;
                margin: 0;
                padding: 20px;
                box-sizing: border-box;
            }
            .widget-button button {
                background: linear-gradient(90deg, #ef4444 0%, #f59e0b 100%) !important;
                border: none !important;
                color: white !important;
                border-radius: 20px !important;
                padding: 8px 25px !important;
                font-weight: bold !important;
            }
            .dropdown select {
                background-color: #2d3748 !important;
                color: white !important;
                padding: 10px !important;
                border-radius: 10px !important;
                width: 100% !important;
                margin-bottom: 20px !important;
            }
        </style>
        '''

        header = widgets.HTML('<h2 style="color: white; text-align: center;">Admin Dashboard</h2>')

        dropdown = widgets.Dropdown(
            options=[(str(idx['Term']), idx) for idx in self.logic.index_data],
            description='Select Index:',
            layout=widgets.Layout(width='400px')
        )

        back_button = widgets.Button(
            description="Back to Search",
            button_style="warning",
            layout=widgets.Layout(width="150px")
        )

        def on_back_clicked(b):
            self.show_search_screen()

        back_button.on_click(on_back_clicked)

        display(widgets.HTML(background_style))
        display(widgets.VBox([header, dropdown, back_button],
                              layout=widgets.Layout(align_items='center')))

    def show_results_screen(self, query, results):
        clear_output()

        display(widgets.HTML('''
        <style>
            body {
                background-color: #1a202c !important;
                color: white !important;
            }
            .widget-button button {
                background: linear-gradient(90deg, #ef4444 0%, #f59e0b 100%) !important;
                border: none !important;
                color: white !important;
                border-radius: 20px !important;
                padding: 8px 25px !important;
                font-weight: bold !important;
            }
        </style>
        '''))

        term_count = sum([r['frequency'] for r in results])
        doc_count = len(results)

        header = widgets.HTML(
            f'''
            <div style="margin: 20px; color: white;">
                <h3>Results for: <i>{query}</i></h3>
                <p>Your search appeared <b>{term_count}</b> times in the following links<br>
                About <b>{doc_count}</b> results</p>
            </div>
            '''
        )

        results_html = []
        for result in results:
            results_html.append(f'''
                <div style="display: flex; justify-content: space-between;
                align-items: center; margin: 15px 0; padding: 10px; color: white;">
                    <a href="{result['page_id']}"
                    style="color: #3b82f6; font-size: 16px; text-decoration: none;">
                        {result['title']}
                    </a>
                    <span style="color: #718096;">Frequency: {result['frequency']}</span>
                </div>
            ''')

        results_widget = widgets.HTML('\n'.join(results_html))

        back_button = widgets.Button(
            description="Back to Search",
            button_style="warning",
            layout=widgets.Layout(width="150px")
        )

        def on_back_clicked(b):
            self.show_search_screen()

        back_button.on_click(on_back_clicked)
        display(widgets.VBox([header, results_widget, back_button]))



In [36]:
# Logic init
firebase_url = "https://cloudproject-27420-default-rtdb.firebaseio.com/"
logic = SearchLogic(firebase_url)

# Style init
ui = SearchUI(logic)
ui.show_search_screen()

HTML(value='\n        <style>\n            body {\n                background-color: #1a202c !important;\n    …

Button(button_style='warning', description='Admin Page', layout=Layout(width='auto'), style=ButtonStyle())

HTML(value='\n        <div style="text-align: center; margin: 40px auto;">\n            <div style="width: 150…